<a href="https://colab.research.google.com/github/rodmorx/DSP_DT_WS24/blob/main/DSProjekt_DT_WS24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Sience Projekt WS24 - Dream Team

In [ ]:
pip install news-please

In [ ]:
from newsplease import NewsPlease
import pandas as pd
import urllib.parse
## Can You READ THIS DREAM TEAM

In [ ]:
# Function to extract the domain (i.e., the newspaper/source) from the URL
def extract_newspaper(url):
    parsed_url = urllib.parse.urlparse(url)
    domain = parsed_url.netloc
    return domain.replace('www.', '')  # Clean up 'www.' if present

In [ ]:
# Function to scrape an article from a URL and return relevant information
def scrape_article(url):
    try:
        article = NewsPlease.from_url(url)
        return {
            'title': article.title,
            'authors': article.authors,
            'date_publish': article.date_publish,
            'maintext': article.maintext,
            'url': url,
            'newspaper': extract_newspaper(url)  # Add newspaper information
        }
    except Exception as e:
        # Return None if there's any error (e.g., 404, timeout)
        print(f"Error scraping {url}: {e}")
        return None

In [ ]:
#Function to scrape multicle articles and return a dataframe with relevant information
def scrape_articles(url_list):
    # Initialize an empty list to store scraped article data
    articles_data = []

    # Loop through each URL and scrape the article
    for url in url_list:
        article_data = scrape_article(url)
        if article_data:
            articles_data.append(article_data)

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(articles_data)

    return df

In [ ]:
# Test Usage
urls = [
    "https://www.abendblatt.de/hamburg/hamburg-mitte/article407529183/elbtower-kauft-tuerkische-firma-hamburgs-kuenftiges-wahrzeichen.html",
    "https://www.sueddeutsche.de/politik/israel-hisbollah-libanon-bank-gold-geld-krankenhaus-lux.TxxwzAtVsoRuvgqkHfMiuy",
    "https://www.morgenpost.de/politik/article407533911/gruene-verbieten-hunde-wie-csu-zu-diesem-schwachsinn-kommt.html"
]

# Scrape articles and create a DataFrame
articles_df = scrape_articles(urls)

# Display the dataframe
print(articles_df.head())

                                               title            authors  \
0  Elbtower: Kauft türkische Firma Hamburgs künft...   [Ulrich Gaßdorf]   
1   Israels Armee sucht den Goldschatz der Hisbollah    [Bernd Dörries]   
2  Grüne verbieten Hunde? Wie CSU zu diesem „Schw...  [Daniel Weidmann]   

         date_publish                                           maintext  \
0 2024-10-24 12:25:16  Hamburg. Bekanntes Unternehmen aus Istanbul so...   
1 2024-10-22 14:56:01  Der Direktor des Krankenhauses, ein Parlaments...   
2 2024-10-24 11:18:36  Berlin. Der CSU-Generalsekretär behauptet, die...   

                                                 url        newspaper  
0  https://www.abendblatt.de/hamburg/hamburg-mitt...    abendblatt.de  
1  https://www.sueddeutsche.de/politik/israel-his...  sueddeutsche.de  
2  https://www.morgenpost.de/politik/article40753...    morgenpost.de  


In [ ]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         3 non-null      object        
 1   authors       3 non-null      object        
 2   date_publish  3 non-null      datetime64[ns]
 3   maintext      3 non-null      object        
 4   url           3 non-null      object        
 5   newspaper     3 non-null      object        
dtypes: datetime64[ns](1), object(5)
memory usage: 272.0+ bytes


In [ ]:
articles_df.head()

,title,authors,date_publish,maintext,url,newspaper
0,Elbtower: Kauft türkische Firma Hamburgs künft...,[Ulrich Gaßdorf],2024-10-24 12:25:16,Hamburg. Bekanntes Unternehmen aus Istanbul so...,https://www.abendblatt.de/hamburg/hamburg-mitt...,abendblatt.de
1,Israels Armee sucht den Goldschatz der Hisbollah,[Bernd Dörries],2024-10-22 14:56:01,"Der Direktor des Krankenhauses, ein Parlaments...",https://www.sueddeutsche.de/politik/israel-his...,sueddeutsche.de
2,Grüne verbieten Hunde? Wie CSU zu diesem „Schw...,[Daniel Weidmann],2024-10-24 11:18:36,"Berlin. Der CSU-Generalsekretär behauptet, die...",https://www.morgenpost.de/politik/article40753...,morgenpost.de


In [ ]:
print(articles_df.loc[0, 'maintext'])

Hamburg. Bekanntes Unternehmen aus Istanbul soll zu den letzten drei Interessenten gehören. Welche Hamburger ebenfalls auf den Zuschlag hoffen.


In [1]:
import newspaper_scraper as nps

ModuleNotFoundError: No module named 'newspaper_scraper'

In [ ]:
with nps.DeBild(db_files='articles_debild.db') as news:
  news.index_articles_by_date_range('2023-09-01', '2023-09-30') #fron 1st to 30th september 2023
  news.scrape_public_articles()
  #it's also possible to apply spacy's nlp function directly with the scraping library
  #news.nlp()